### SQL Tableau Dashboards for Business Intelligence / Advanced Data Analysis

By Michael Stephenson<p/>




Import Python libraries

In [1]:
import pandas as pd
import sqlite3
import sql

import warnings
warnings.filterwarnings('ignore')

Create a connection to the sqlite3 library.

In [2]:
data_db = sqlite3.connect('example_sql.db')
cursor = data_db.cursor()

Load SQL into iPython

In [3]:
%load_ext sql

Create Jupyter database

In [4]:
%sql sqlite:///example_sql.db

Import churn_raw_data

In [5]:
churn_raw = pd.read_csv('churn_raw_data.csv')

Load churn_raw_data into SQLite Jupyter Database 

In [6]:
churn_raw.to_sql('churn_raw', data_db)

Import WA_Fn-UseC_-Telco-Customer-Churn

In [7]:
Telco_Customer = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

Load WA_Fn-UseC_-Telco-Customer-Churn into SQLite Jupyter Database

In [8]:
Telco_Customer.to_sql('Telco_Customer', data_db)

Display table names

In [10]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///example_sql.db
Done.


name
churn_raw
Telco_Customer


Count rows for churn_raw

In [11]:
%sql SELECT COUNT(*) AS number_rows From churn_raw



 * sqlite:///example_sql.db
Done.


number_rows
10000


Count rows for Telco_Customer

In [12]:
%sql SELECT COUNT(*) AS number_rows From Telco_Customer

 * sqlite:///example_sql.db
Done.


number_rows
7043


Clean_data table is from the churn_data table.

In [14]:
%%sql

CREATE TABLE clean_data(
    Customer_id CHAR(50) PRIMARY KEY,
    Tenure         FLOAT,
    MonthlyCharge FLOAT,
    Churn CHAR(50),
    State CHAR(50),
    Population INT) ;
    
INSERT INTO clean_data SELECT Customer_id, Tenure, MonthlyCharge, Churn, State, Population FROM churn_raw; 



SELECT * FROM clean_data  LIMIT 10;



 * sqlite:///example_sql.db
(sqlite3.OperationalError) table clean_data already exists
[SQL: CREATE TABLE clean_data( Customer_id CHAR(50) PRIMARY KEY,
    Tenure         FLOAT,
    MonthlyCharge FLOAT,
    Churn CHAR(50),
    State CHAR(50),
    Population INT) ;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


### Count Null Vaules

In [15]:
%%sql sqlite://
SELECT COUNT(*)-count(State) AS NullVaules
FROM clean_data
WHERE State IS NULL;

Done.


NullVaules
0


In [16]:
%%sql sqlite://
SELECT COUNT(*)-count(Population) AS NullVaules
FROM clean_data
WHERE Population IS NULL;

Done.


NullVaules
0


In [17]:
%%sql sqlite://
SELECT COUNT(*)-count(Tenure) AS NullVaules
FROM clean_data
WHERE Tenure IS NULL;

Done.


NullVaules
931


In [18]:
%%sql sqlite://
SELECT COUNT(*)-count(MonthlyCharge) AS NullVaules
FROM clean_data
WHERE MonthlyCharge IS NULL;

Done.


NullVaules
0


In [19]:
%%sql sqlite://
SELECT COUNT(*)-count(Churn) AS NullVaules
FROM clean_data
WHERE Churn IS NULL;

Done.


NullVaules
0


Drop missing values

In [20]:
%%sql sqlite://
DELETE FROM clean_data 
WHERE Tenure IS NULL;

SELECT * FROM clean_data LIMIT 10;

931 rows affected.
Done.


Customer_id,Tenure,MonthlyCharge,Churn,State,Population
K409198,6.795512947,171.4497621,No,AK,38
S120509,1.156680997,242.9480155,Yes,MI,10446
K191035,15.75414408,159.4403984,No,OR,3735
D90850,17.08722662,120.2494934,No,CA,13863
K662701,1.670971726,150.7612159,Yes,TX,11352
W303516,7.000993555,184.4015581,No,GA,17701
U335188,13.23677381,200.0648859,Yes,TN,2535
M716771,8.220686373,118.3668439,No,FL,17351
I676080,3.422086139,163.0052796,No,OH,20193
J980369,19.26726194,177.451599,No,PA,555


Add the variable Gross_Profits to clean_data

In [22]:
%%sql sqlite://
ALTER TABLE clean_data ADD Gross_Profits FLOAT NULL;
SELECT * FROM clean_data LIMIT 10;

(sqlite3.OperationalError) duplicate column name: Gross_Profits
[SQL: ALTER TABLE clean_data ADD Gross_Profits FLOAT NULL;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


Update Gross_Profits with sum of MonthlyCharge

In [23]:
%%sql sqlite://
UPDATE 
    clean_data 
SET 
    Gross_Profits= (SELECT SUM(MonthlyCharge) FROM clean_data); 


SELECT Gross_Profits FROM clean_data LIMIT 10;

9069 rows affected.
Done.


Gross_Profits
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692
1580515.3218268692


In [24]:
%%sql sqlite://
ALTER TABLE clean_data ADD Avg_Cost FLOAT NULL;
SELECT * FROM clean_data LIMIT 10;

Done.
Done.


Customer_id,Tenure,MonthlyCharge,Churn,State,Population,Gross_Profits,Avg_Cost
K409198,6.795512947,171.4497621,No,AK,38,1580515.3218268692,None
S120509,1.156680997,242.9480155,Yes,MI,10446,1580515.3218268692,None
K191035,15.75414408,159.4403984,No,OR,3735,1580515.3218268692,None
D90850,17.08722662,120.2494934,No,CA,13863,1580515.3218268692,None
K662701,1.670971726,150.7612159,Yes,TX,11352,1580515.3218268692,None
W303516,7.000993555,184.4015581,No,GA,17701,1580515.3218268692,None
U335188,13.23677381,200.0648859,Yes,TN,2535,1580515.3218268692,None
M716771,8.220686373,118.3668439,No,FL,17351,1580515.3218268692,None
I676080,3.422086139,163.0052796,No,OH,20193,1580515.3218268692,None
J980369,19.26726194,177.451599,No,PA,555,1580515.3218268692,None


Average Cost to clean_data customers

In [25]:
%%sql sqlite://
    
UPDATE 
    clean_data 
SET 
    Avg_Cost= (SELECT AVG(MonthlyCharge) FROM clean_data); 


SELECT Avg_Cost FROM clean_data LIMIT 10;

9069 rows affected.
Done.


Avg_Cost
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805
174.27669222922805


Telco_Clean table is from the Telco_Customer table.

In [26]:
%%sql 

CREATE TABLE Telco_Clean(
    customerID CHAR(50) PRIMARY KEY,
    tenure         FLOAT,
    MonthlyCharges FLOAT,
    Churn CHAR(50));
    
INSERT INTO Telco_Clean SELECT customerID, tenure, MonthlyCharges, Churn FROM Telco_Customer; 



SELECT * FROM Telco_Clean  LIMIT 10;

 * sqlite:///example_sql.db
Done.
7043 rows affected.
Done.


customerID,tenure,MonthlyCharges,Churn
7590-VHVEG,1.0,29.85,No
5575-GNVDE,34.0,56.95,No
3668-QPYBK,2.0,53.85,Yes
7795-CFOCW,45.0,42.3,No
9237-HQITU,2.0,70.7,Yes
9305-CDSKC,8.0,99.65,Yes
1452-KIOVK,22.0,89.1,No
6713-OKOMC,10.0,29.75,No
7892-POOKP,28.0,104.8,Yes
6388-TABGU,62.0,56.15,No


### Count Null Vaules

In [27]:
%%sql sqlite://
SELECT COUNT(*) -count(tenure) AS NullVaules
FROM Telco_Clean
WHERE tenure IS NULL;

Done.


NullVaules
0


In [28]:
%%sql sqlite://
SELECT COUNT(*) -count(MonthlyCharges) AS NullVaules
FROM Telco_Clean
WHERE MonthlyCharges IS NULL;

Done.


NullVaules
0


In [29]:
%%sql sqlite://
SELECT COUNT(*) -count(Churn) AS NullVaules
FROM Telco_Clean
WHERE Churn IS NULL;

Done.


NullVaules
0


### SQL custom queries to data.

#### Select all variables from table.

In [30]:
%%sql sqlite://
    
SELECT * FROM clean_data LIMIT 10;

Done.


Customer_id,Tenure,MonthlyCharge,Churn,State,Population,Gross_Profits,Avg_Cost
K409198,6.795512947,171.4497621,No,AK,38,1580515.3218268692,174.27669222922805
S120509,1.156680997,242.9480155,Yes,MI,10446,1580515.3218268692,174.27669222922805
K191035,15.75414408,159.4403984,No,OR,3735,1580515.3218268692,174.27669222922805
D90850,17.08722662,120.2494934,No,CA,13863,1580515.3218268692,174.27669222922805
K662701,1.670971726,150.7612159,Yes,TX,11352,1580515.3218268692,174.27669222922805
W303516,7.000993555,184.4015581,No,GA,17701,1580515.3218268692,174.27669222922805
U335188,13.23677381,200.0648859,Yes,TN,2535,1580515.3218268692,174.27669222922805
M716771,8.220686373,118.3668439,No,FL,17351,1580515.3218268692,174.27669222922805
I676080,3.422086139,163.0052796,No,OH,20193,1580515.3218268692,174.27669222922805
J980369,19.26726194,177.451599,No,PA,555,1580515.3218268692,174.27669222922805


In [31]:
%%sql sqlite://
    
SELECT * FROM Telco_Clean LIMIT 10;

Done.


customerID,tenure,MonthlyCharges,Churn
7590-VHVEG,1.0,29.85,No
5575-GNVDE,34.0,56.95,No
3668-QPYBK,2.0,53.85,Yes
7795-CFOCW,45.0,42.3,No
9237-HQITU,2.0,70.7,Yes
9305-CDSKC,8.0,99.65,Yes
1452-KIOVK,22.0,89.1,No
6713-OKOMC,10.0,29.75,No
7892-POOKP,28.0,104.8,Yes
6388-TABGU,62.0,56.15,No


#### Arrange data according to churn type, average tenture, and average monthly charge.

In [32]:
%%sql sqlite://
    
SELECT Churn, round(AVG(tenure),2) AS tenure,
round(AVG(MonthlyCharge),2) AS MonthlyCharges
FROM clean_data 
GROUP BY Churn;


Done.


Churn,tenure,MonthlyCharges
No,42.25,164.57
Yes,13.22,200.94


In [33]:
%%sql sqlite://
    
SELECT Churn, round(SUM(tenure),2) AS tenure,
round(AVG(MonthlyCharges),2) AS MonthlyCharges
FROM Telco_Clean 
GROUP BY Churn;

Done.


Churn,tenure,MonthlyCharges
No,194387.0,61.27
Yes,33603.0,74.44


#### Run SELECT statement to produce a dataframe containing all data. 

In [34]:
import pandas as pd
# read in your SQL query results using pandas
dataframe = pd.read_sql("""
            SELECT *
            FROM clean_data
            """,data_db)
dataframe.head()

,Customer_id,Tenure,MonthlyCharge,Churn,State,Population,Gross_Profits,Avg_Cost
0,K409198,6.795513,171.449762,No,AK,38,1.580515e+06,174.276692
1,S120509,1.156681,242.948015,Yes,MI,10446,1.580515e+06,174.276692
2,K191035,15.754144,159.440398,No,OR,3735,1.580515e+06,174.276692
3,D90850,17.087227,120.249493,No,CA,13863,1.580515e+06,174.276692
4,K662701,1.670972,150.761216,Yes,TX,11352,1.580515e+06,174.276692


#### Sources

BlastChar (n.d) Telco Customer Churn Retrieved from https://www.kaggle.com/blastchar/telco-customer-churn <p/>

royalosyin (2021, April 28) Practice-SQL-with-SQLite-and-Jupyter-Notebook Retrieved from https://github.com/royalosyin/Practice-SQL-with-SQLite-and-Jupyter-Notebook <p/>

Western Governor’s University. (June 2021). D206 Data Cleaning_Churn Data Consideration and Dictionary [PDF file]. Retrieved from https://access.wgu.edu/ASP3/aap/content/kgj47f8gj49f8du49d3k.html <p/>

